# GLM 的函数调用从入门到精通

## 概述

GLM 的函数调用功能为开发者提供了将模型与外部函数库或 API 连接的强大能力。通过这个功能,模型允许我们提供一组预定义的工具，让模型在对话过程中调用这些工具。

你可以把它想象成一个**预先构建的工具箱**，里面装有各种工具函数，每个工具函数都有特定的功能和用途，**并可以实际运行**。



## 函数调用功能简介

ChatGLM 的函数调用功能为开发者提供了一种将模型与外部功能无缝集成的方式。通过这个功能,模型可以在对话过程中生成外部函数的调用参数,从而帮助完成各种复杂的任务。这一功能极大地扩展了模型的能力,让它能够:

1. **增强推理和实时效果**:模型可以调用外部函数获取精确的信息,从而做出更加智能的决策。比如可以调用天气预报 API 获取当地天气情况,或者查询航班信息等。
2. **执行外部操作**:模型可以触发外部操作,如数据库操作、系统指令执行、事件触发等。这使得模型能够直接与系统交互,而不仅仅局限于自然语言交流。
3. **提高交互效率**:模型可以自主决定何时需要调用函数,并生成所需的参数。这样可以大大提高对话的流畅性和用户体验。



## 理解函数调用机制与全流程

在使用函数调用功能时,理解其工作原理至关重要。整体的工作流程如下:

1. **定义可调用的函数**:开发者使用 `tools` 参数向模型提供函数的定义,包括名称、描述和参数。
2. **与模型交互（API）**:用户通过聊天界面提出问题,模型根据上下文决定是否需要调用外部函数。
3. **模型生成函数调用**:如果模型认为需要调用函数,它会生成函数名称和所需的参数。

----------可在这里结束，也可以继续把本地执行函数的结果再返回给大模型-------------------------------

4. **执行函数**:开发者使用模型提供的参数调用实际的外部函数,并获取结果。
5. **返回结果给模型**:将函数执行结果作为新的消息传递给模型。
6. **模型继续对话（API）**:模型基于函数调用的结果,生成最终的回复返回给用户。

---

## 定义外部函数的“注释”

### 使用 `tools` 参数

`tools` 是 ChatGLM API 中的可选参数,用于向模型提供可调用的外部函数定义。通过这个参数,模型能够识别可用的函数,并在需要时生成调用。

`tools` 参数是一个函数列表,每个函数都包含以下字段:

- `type`: 固定设置为 `"function"`。
- `function`: 一个对象,包含函数的详细信息。

### 描述函数功能

在 `function` 对象中,需要提供以下信息（外层）:

- `name`: 函数名称,只能包含字母、数字、下划线和中横线。
- `description`: 对函数功能的详细描述,模型会根据这段描述决定如何调用函数。
- `parameters`: 使用 JSON Schema 定义函数所需的参数。


每一个函数中再使用 JSON Schema 准确地定义函数接受的参数（内层）:

- `type`: 数据类型,一般为 `"object"`。
- `properties`: 一个对象,定义每个参数的名称、类型和描述。
- `required`: 一个数组,列出必须提供的参数名称。


通过这种方式,模型就能理解(类似函数注释):
- 函数的用途和能力
- 需要什么输入参数
- 如何正确调用函数

**示例:**

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "根据城市名称获取当前天气信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "description": "城市名称,例如:北京",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期,可选,格式为 YYYY-MM-DD",
                        "type": "string"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

在这个示例中,我们定义了一个名为 `get_weather` 的函数,它接受 `city` 和可选的 `date` 参数,用于获取指定城市和日期的天气信息。

---

## 与模型交互

### 触发函数调用

在对话过程中,模型会根据用户的输入和预先定义的函数,判断是否需要调用外部函数。当模型认为需要调用函数时,它会在响应中返回函数名称和生成的参数。

**示例:**

在这个示例中,模型可能会生成一个函数调用,表示需要调用 `get_weather` 函数,并提供参数 `"city": "北京", "date": "2022-12-12"`。

### 控制函数选择（`tool_choice`）

`tool_choice` 参数允许开发者控制模型如何选择函数调用，目前仅支持 auto 模式。（只能说？聊胜于无）:

- `auto`(默认): 模型自行决定是否调用函数。
- `none`: 强制模型不返回任何函数调用,仅给出纯文本回复。

这里需要了解的是OPENAI的可选值："auto","required", "none", {"type": "function","function": {"name": "function_name"}}

GLM目前这部分还有待继续开发完全。

**示例:**

In [76]:
from zhipuai import ZhipuAI

# client = ZhipuAI(api_key="YOUR_API_KEY")
client = ZhipuAI()

In [77]:
messages = [
    {"role": "user", "content": "请告诉我北京的天气如何?"}
]
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

In [78]:
print(response)

Completion(model='glm-4-plus', created=1732188152, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_-9221298494868164792', function=Function(arguments='{"city": "北京"}', name='get_weather'), type='function', index=0)]))], request_id='202411211922311f49ccc884044a5a', id='202411211922311f49ccc884044a5a', usage=CompletionUsage(prompt_tokens=195, completion_tokens=10, total_tokens=205))


模型本身永远不会实际执行函数，模型只是生成可用于调用函数的参数，然后我们的本地代码可以选择如何处理这些参数，

当模型决定调用函数时,它会在响应中返回 `tool_calls` 字段,包含函数名称和参数。


In [ ]:
{
    "choices": [
        {
            "message": {
                "role": "assistant",
                "content": None,
                "tool_calls": [
                    {
                        "id": "call_123456",
                        "function": {
                            "name": "get_weather",
                            "arguments": '{"city": "北京", "date": "2023-04-20"}'
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ]
}

In [79]:
if response.choices[0].message.tool_calls:
    function_call = response.choices[0].message.tool_calls[0].function
    print(function_call.name)
    print(function_call.arguments)

get_weather
{"city": "北京"}


在这个示例中,模型生成了一个 `get_weather` 函数调用,并提供了 `city` 和 `date` 两个参数，有些时候以上就可以截止了

什么时候适合只使用工具调用的前3步，而不用完整的工具调用呢？:

**JSON输出场景**
- 当你只需要模型返回符合特定schema的JSON输出时
- 例如记录摘要、情感分析等结构化输出场景

**实时处理场景** 
- 实时内容审核
- 数据提取和标注任务
- 代码补全等需要快速响应的场景

**独立处理场景**
- 当工具的输出不需要模型进一步解释或加工
- 当你希望自己的应用程序处理工具的输出结果

在这些场景中,我们主要使用tool use来:
1. 理解用户意图
2. 选择合适的工具
3. 构造正确的参数

这种方式可以:
- 减少API调用次数，降低延迟，只借助大模型的意图识别及取值能力，让应用程序更灵活地处理结果


### 执行模型生成的函数调用

为了完成整个函数调用流程,开发者需要执行以下步骤:

1. **解析模型生成的参数**:从 `arguments` 字段获取参数,并解析为字典。
2. **调用实际的函数**:使用解析的参数调用预先定义的外部函数。
3. **将结果返回给模型**:将函数执行结果作为新的消息,角色为 `"tool"`的类型,传递给模型。

**示例代码:**

In [29]:
import json

def parse_function_call(response, messages, client, tools):
    try:
        # 提取工具调用和相关数据
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        messages.append(response.choices[0].message.model_dump())
        # 定义支持的函数处理逻辑
        function_map = {
            "get_weather": get_weather,
            # 可以扩展更多函数
        }
        
        if function_name in function_map:
            # 调用对应的函数并获取结果
            result = function_map[function_name](**arguments)
            messages.append({
                "role": "tool",
                "content": json.dumps(result, ensure_ascii=False),
                "tool_call_id": tool_call.id
            })
            
            # 再次调用模型
            return client.chat.completions.create(
                model="glm-4-plus",
                messages=messages,
                tools=tools
            )
        else:
            raise ValueError(f"Unsupported function: {function_name}")
    except Exception as e:
        print(f"Error while parsing function call: {e}")
        return None


In [6]:
parse_function_call(response, messages, client, tools)

Error while parsing function call: name 'get_weather' is not defined


适合完整6步的场景:

**需要多工具链式调用**
- 当任务需要多个工具按顺序配合完成
- 后续工具的输入依赖于前一个工具的输出，要协调多个工具的结果

**需要解释和分析结果**
- 当需要将技术结果，复杂数据转化为用户友好的解释，提供建议或见解

而3步流程适用于:

**结构化输出场景**
- 当只需要JSON格式的输出
- 数据提取和标注任务
- 不需要进一步解释的分析结果

主要区别:
- 6步流程适合需要模型智能处理和整合结果的复杂任务
- 3步流程适合直接使用工具输出的简单任务

In [30]:
import json

def get_weather(city):
    """
    正式业务场景中可能会请求一些实时的API得到如下的结果
    """
    
    return {
        "city": city,
        "date": "明天",
        "temperature": 22,
        "weather": "小雨",
        "humidity": 85,
        "wind": "东北风3级"
    }

In [31]:
messages

[{'role': 'user', 'content': '请告诉我北京的天气如何?'}]

In [32]:
new_message = {"role": "system", "content": "根据天气信息，请你根据现实准确专业的给出出行，穿衣，活动建议"}

# 在第一条位置添加新消息
messages.insert(0, new_message)
messages

[{'role': 'system', 'content': '根据天气信息，请你根据现实准确专业的给出出行，穿衣，活动建议'},
 {'role': 'user', 'content': '请告诉我北京的天气如何?'}]

In [33]:
twice_response=parse_function_call(response, messages, client, tools)

In [34]:
print(twice_response.choices[0].message.content)

根据北京的天气信息，以下是出行、穿衣和活动的建议：

**出行建议：**
1. 由于有小雨，建议携带雨具，如雨伞或雨衣。
2. 注意路面湿滑，步行和驾驶时需谨慎。
3. 东北风3级，出行时注意防风保暖。

**穿衣建议：**
1. 温度为22°C，建议穿着长袖上衣和轻薄的外套。
2. 由于湿度较高，衣物选择透气性好的材质会更舒适。
3. 考虑到有小雨，可以选择防水或速干材质的衣物。

**活动建议：**
1. 室外活动尽量选择雨停的时段，或选择室内活动以避免淋雨。
2. 高湿度环境下，避免进行剧烈运动，以免身体不适。
3. 可以选择参观博物馆、艺术馆等室内场所，或进行一些轻松的室内娱乐活动。

请注意，以上建议是基于当前的天气状况，实际情况可能会有所变化，出行前建议再次查看最新天气预报。


In [40]:
messages.append(twice_response.choices[0].message.model_dump())

Message 1: 用户提出关于北京天气的疑问。
Message 2: 助手识别出需要外部数据，调用get_weather函数获取天气信息。
Message 3: 天气工具返回北京的天气数据，包括温度、天气状况等详细信息。
Message 4: 助手根据获取的天气信息，生成专业的出行、穿衣和活动建议，并回复给用户。

In [41]:
messages

[{'role': 'system', 'content': '根据天气信息，请你根据现实准确专业的给出出行，穿衣，活动建议'},
 {'role': 'user', 'content': '请告诉我北京的天气如何?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_-9221303305233519401',
    'function': {'arguments': '{"city": "北京"}', 'name': 'get_weather'},
    'type': 'function',
    'index': 0}]},
 {'role': 'tool',
  'content': '{"city": "北京", "date": "明天", "temperature": 22, "weather": "小雨", "humidity": 85, "wind": "东北风3级"}',
  'tool_call_id': 'call_-9221303305233519401'},
 {'content': '根据北京的天气信息，以下是出行、穿衣和活动的建议：\n\n**出行建议：**\n1. 由于有小雨，建议携带雨具，如雨伞或雨衣。\n2. 注意路面湿滑，步行和驾驶时需谨慎。\n3. 东北风3级，出行时注意防风保暖。\n\n**穿衣建议：**\n1. 温度为22°C，建议穿着长袖上衣和轻薄的外套。\n2. 由于湿度较高，衣物选择透气性好的材质会更舒适。\n3. 考虑到有小雨，可以选择防水或速干材质的衣物。\n\n**活动建议：**\n1. 室外活动尽量选择雨停的时段，或选择室内活动以避免淋雨。\n2. 高湿度环境下，避免进行剧烈运动，以免身体不适。\n3. 可以选择参观博物馆、艺术馆等室内场所，或进行一些轻松的室内娱乐活动。\n\n请注意，以上建议是基于当前的天气状况，实际情况可能会有所变化，出行前建议再次查看最新天气预报。',
  'role': 'assistant',
  'tool_calls': None}]

注意两点，一个是消息的添加要全面，即要添加函数决定调用函数的消息和本地执行的结果,注意顺序，GLM并不会报错，但gpt会校验role为tool的消息前有没有tool call的消息，同时注意tool id。

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241031105033.png)

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241121165531.png)

在把工具调用的结果消息放进message里时，我们也是要存字符串进去的，所以要用json.dump把字典结构转换为 JSON 格式的字符串.总结：

- 接收参数时：JSON字符串 -> Python对象（方便处理）用json.loads
- 返回结果时：Python对象 -> JSON字符串（方便传输）用json.dump

【实战】利用维基百科为写作助手提供信息支持

1. 定义维基百科搜索功能:
编写 `get_article()` 函数,可以根据搜索词获取维基百科页面的内容。

2. 编写工具定义:
定义 `article_search_tool` 字典,描述了 `get_article()` 函数的元数据信息,包括函数名、描述和参数定义。这样 GLM 就可以知道如何调用这个工具了。

3. 向 GLM 发送 API 请求:
编写发送请求到 GLM 的代码,将对话消息、可用工具和工具选择传递给 GLM。这是与 GLM 进行交互的核心步骤。

4. GLM 决策并返回工具名与入参:
成功地解析 GLM 的响应,获取了推荐的工具函数名和参数。这一步是连接 GLM 和本地执行函数的关键。

5. 本地执行工具函数,得到结果:
编写了执行 `get_article()` 函数的代码,并将结果转换为可以返回给 GLM 的格式。这一步完成了工具函数的本地执行。

6. 向 GLM 提供本地执行的工具结果:
最后,将工具函数的执行结果添加到对话历史中,并再次发送请求给 GLM,让它基于新的信息生成最终的响应。这样就形成了一个完整的交互循环。


1. 定义我们的维基百科搜索功能

In [42]:
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl


In [45]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0] #通常，第一个结果是与搜索词最相关的页面。
    page = wikipedia.page(first_result, auto_suggest=False) #获取与该标题对应的页面对象，然后通过 page.content 获取该页面的内容。
    return page.content

article = get_article("特朗普")
print(article[:500]) 

Transcription into Chinese characters is the use of traditional or simplified Chinese characters to phonetically transcribe the sound of terms and names of foreign words to the Chinese language. Transcription is distinct from translation into Chinese whereby the meaning of a foreign word is communicated in Chinese. Since English classes are now standard in most secondary schools, it is increasingly common to see foreign names and terms left in their original form in Chinese texts. However, for m


2. 编写工具定义

In [80]:
article_search_tool = [
    {
        "type": "function",
        "function":{
            "name": "get_article",
            "description": "A tool to retrieve an up to date Wikipedia article,notice，should be English.",
            "parameters": {
             "type": "object",
             "properties": {
                 "search_term": {
                     "type": "string",
                     "description": "The search term to find a wikipedia article by title,notice，should be English"
                 },
             },
             "required": ["search_term"]
         }
     }
}]

3. 向GLM发送API请求，携带工具与问题 

In [81]:
messages = [{"role": "user", "content": "2024年的影响最大的事件"}]

In [82]:
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
    tools=article_search_tool,
)  

4. GLM决策并返回：工具名与入参

In [83]:
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0].function
    print("工具函数的名字:",tool_call.name)
    print("工具函数的输入:",tool_call.arguments) 

工具函数的名字: get_article
工具函数的输入: {"search_term": "2024 significant events"}


5. 本地执行工具函数,得到结果,同时记得要添加**两次**消息到历史对话中

In [84]:
if tool_call.name == "get_article":
    arguments = json.loads(tool_call.arguments)
    search_term = arguments.get("search_term")
    wiki_result = get_article(search_term)
    print(f"搜索到了 {search_term}")
    print("维基百科的原文:")
    print(wiki_result[:500])

搜索到了 2024 significant events
维基百科的原文:
This article is a list of significant events that occur in aviation in 2024.


== Events ==


=== January ===
2 January
A runway collision at Haneda Airport in Tokyo occurred when Japan Airlines Flight 516, operated by an Airbus A350-900 arriving from Sapporo, collided with a Japan Coast Guard aircraft and both aircraft caught on fire. This resulted in the complete destruction of both aircraft. All 367 passengers and 12 crew members of the Airbus were evacuated. There were six occupants on board


现在我们已经在本地执行了函数调用，我们需要将此函数调用的结果提供回API

In [ ]:
import json

messages = [ {"role": "system", "content": "根据天气信息，请你根据现实准确专业的给出出行，穿衣，活动建议"},
             {"role": "user", "content": "北京今天天气如何？"}]

stream = client.chat.completions.create(
    model="glm-4-alltools",
    messages=messages,
    tools=tools,
    stream=True
) 
function_call_content = ''
function_name = ''
for chunk in stream:
    # 检查是否有函数调用的信息
    if chunk.choices[0].delta.tool_calls:
        for tool_call in chunk.choices[0].delta.tool_calls:
            # 获取函数名
            if tool_call.function.name:
                function_name = tool_call.function.name
            # 拼接函数参数
            if tool_call.function.arguments:
                function_call_content += tool_call.function.arguments
    # 检查是否完成了函数调用
    if chunk.choices[0].finish_reason == 'tool_calls':
        # 解析函数参数
        try:
            function_call_content = function_call_content.split('```')[-1].strip()
            args = json.loads(function_call_content)

In [66]:
response.choices[0].message.model_dump()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_-9221298597947443131',
   'function': {'arguments': '{"search_term": "2024 significant events"}',
    'name': 'get_article'},
   'type': 'function',
   'index': 0}]}

In [85]:
import json

def parse_function_call(response, messages, client):
    try:
        # 提取工具调用和相关数据
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        messages.append(response.choices[0].message.model_dump())
        # 定义支持的函数处理逻辑
        function_map = {
            "get_weather": get_weather,
            "get_article": get_article
            # 可以扩展更多函数
        }
        
        if function_name in function_map:
            # 调用对应的函数并获取结果
            result = function_map[function_name](**arguments)
            messages.append({
                "role": "tool",
                "content": json.dumps(result, ensure_ascii=False),
                "tool_call_id": tool_call.id
            })
            
            # 再次调用模型
            return client.chat.completions.create(
                model="glm-4-plus",
                messages=messages,
                # tools=tools
            )
        else:
            raise ValueError(f"Unsupported function: {function_name}")
    except Exception as e:
        print(f"Error while parsing function call: {e}")
        return None


In [86]:
result = parse_function_call(response, messages, client)

In [87]:
print(result.choices[0].message.content)

2024年的重大事件主要集中在航空领域，其中包括多起飞机事故、航空公司运营变动以及重要的航空展览和订单。以下是一些显著的事件：

1. **1月2日**：东京羽田机场发生跑道碰撞事故，日本航空516航班与一架日本海岸警卫队飞机相撞并起火，造成两架飞机完全损毁。

2. **1月5日**：阿拉斯加航空1282航班在波特兰国际机场起飞后不久发生爆炸性减压，飞机安全返回。

3. **1月16日**：美国司法部阻止捷蓝航空以38亿美元收购精神航空。

4. **1月18日**：阿卡萨航空在WINGS India 2024活动上订购了150架波音737 MAX飞机。

5. **1月24日**：一架俄罗斯伊尔-76M运输机在俄罗斯别尔哥罗德州坠毁，据报道机上有65名乌克兰战俘。

6. **2月6日**：前智利总统塞巴斯蒂安·皮涅拉驾驶的罗宾逊R44直升机失控坠入兰科湖。

7. **2月18日**：塞尔维亚航空324航班在贝尔格莱德尼古拉特斯拉机场起飞时撞上着陆灯，紧急返回时机身出现大洞。

8. **3月12日**：一架俄罗斯伊尔-76军用运输机在别尔哥罗德附近坠毁，机上15人全部遇难。

9. **3月22日**：Boom Supersonic的XB-1超音速原型机进行首次飞行。

10. **4月13日**：伊拉克、约旦、黎巴嫩、叙利亚、科威特和以色列因伊朗对以色列的袭击而关闭领空。

11. **5月19日**：搭载伊朗总统莱西的贝尔212直升机在东阿塞拜疆省瓦尔扎甘附近坠毁，无人生还。

12. **5月21日**：新加坡航空321航班从伦敦飞往新加坡途中遭遇严重颠簸，造成1名乘客死亡，104人受伤。

13. **6月10日**：马拉维空军一架多尼尔228飞机坠毁，机上包括副总统奇利马在内的9人全部遇难。

14. **7月19日**：CrowdStrike事件导致全球5078个航班取消。

15. **8月9日**：沃帕斯航空2283航班的一架ATR 72-500在圣保罗附近坠毁，机上62人全部遇难。

16. **9月1日**：北欧航空在27年后退出星空联盟，加入天合联盟。

17. **9月13日**：超过33,000名波音机械师罢工，停止了波音737、777和767的生产。

18. **10月26日**：捷克航空在运营近101年后停止运营。

19.